In [ ]:
# -*- coding: utf-8 -*-

import os
import tempfile
import numpy as np

from PyQt5.Qt import *
from PyQt5.QtCore import Qt

import md.image3d.python.image3d_io as cio
import md.image3d.python.image3d_tools as ctools
from md.utils.python.file_tools import readlines
from md.segmentation3d.utils.imlist_tools import make_train_imlist_fp, make_test_imlist_fp
import md.viewer.common.view3_cfg as view3_cfg
from md.viewer.common.view3 import View3
from md.segmentation3d.vseg import autoseg_load_model, autoseg_volume


class OptionDialog(QDialog):
    """
    Dialog for displaying option buttons
    """
    def __init__(self, options, fontsize=16):
        
        super(OptionDialog, self).__init__()

        self.selected_id = -1

        font = self.font()
        font.setPointSize(fontsize)

        self.button_ids = {}
        layout = QHBoxLayout()
        for i, option in enumerate(options):
            button = QPushButton(option)
            button.setFont(font)
            button.setSizePolicy(QSizePolicy.Ignored, QSizePolicy.Ignored)
            button.clicked.connect(self.buttonClicked)
            self.button_ids[button] = i
            layout.addWidget(button)

        self.setLayout(layout)

    def buttonClicked(self):
        
        button = self.sender()
        button_id = self.button_ids[button]
        self.selected_id = button_id
        self.accept()


class TestNameDialog(QDialog):

    def __init__(self, fontsize=16):

        super(TestNameDialog, self).__init__()

        self.org_name = ''

        font = self.font()
        font.setPointSize(fontsize)

        layout = QVBoxLayout()

        org_label = QLabel('Image')
        self.org_combobox = QComboBox(self)
        self.org_combobox.setEditable(False)
        self.org_combobox.currentIndexChanged.connect(self.orgIndexChanged)
        org_label.setFont(font)
        self.org_combobox.setFont(font)

        layout1 = QHBoxLayout()
        layout1.addWidget(org_label)
        layout1.addWidget(self.org_combobox)

        layout2 = QHBoxLayout()
        ok_button = QPushButton('OK')
        ok_button.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
        ok_button.clicked.connect(self.accept)
        ok_button.setFont(font)

        cancel_button = QPushButton('Cancel')
        cancel_button.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
        cancel_button.clicked.connect(self.reject)
        cancel_button.setFont(font)

        layout2.addWidget(ok_button)
        layout2.addWidget(cancel_button)

        layout.addLayout(layout1)
        layout.addLayout(layout2)

        self.setLayout(layout)

    def orgIndexChanged(self, _):

        self.org_name = self.org_combobox.currentText()

    def segIndexChanged(self, _):

        self.seg_name = self.seg_combobox.currentText()


class TrainNameDialog(QDialog):

    def __init__(self, fontsize=16):

        super(TrainNameDialog, self).__init__()

        self.org_name = ''
        self.seg_name = ''

        font = self.font()
        font.setPointSize(fontsize)

        layout = QVBoxLayout()

        layout1 = QHBoxLayout()
        org_label = QLabel('Image')
        self.org_combobox = QComboBox(self)
        self.org_combobox.setEditable(False)
        self.org_combobox.currentIndexChanged.connect(self.orgIndexChanged)

        seg_label = QLabel('Seg')
        self.seg_combobox = QComboBox(self)
        self.seg_combobox.setEditable(False)
        self.seg_combobox.currentIndexChanged.connect(self.segIndexChanged)

        org_label.setFont(font)
        self.org_combobox.setFont(font)
        seg_label.setFont(font)
        self.seg_combobox.setFont(font)

        layout1.addWidget(org_label)
        layout1.addWidget(self.org_combobox)
        layout1.addWidget(seg_label)
        layout1.addWidget(self.seg_combobox)

        layout2 = QHBoxLayout()
        ok_button = QPushButton('OK')
        ok_button.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
        ok_button.clicked.connect(self.accept)
        ok_button.setFont(font)

        cancel_button = QPushButton('Cancel')
        cancel_button.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
        cancel_button.clicked.connect(self.reject)
        cancel_button.setFont(font)

        layout2.addWidget(ok_button)
        layout2.addWidget(cancel_button)

        layout.addLayout(layout1)
        layout.addLayout(layout2)

        self.setLayout(layout)

    def orgIndexChanged(self, _):

        self.org_name = self.org_combobox.currentText()

    def segIndexChanged(self, _):

        self.seg_name = self.seg_combobox.currentText()


def read_train_list(lines):

    imlist, seglist = [], []
    casenum = int(lines[0])

    for i in range(casenum):
        im_path = lines[1 + i * 2]
        seg_path = lines[2 + i * 2]

        if not os.path.isfile(im_path):
            QMessageBox.critical(None, 'File Not Found', im_path)
            return [], []
        if not os.path.isfile(seg_path):
            QMessageBox.critical(None, 'File Not Found', seg_path)
            return [], []

        imlist.append(im_path)
        seglist.append(seg_path)

    return imlist, seglist


def read_test_list(lines):

    imlist, seglist = [], []
    casenum = int(lines[0])

    for i in range(casenum):
        idx = lines[i+1].index(' ')
        assert idx > 0
        im_path = lines[i+1][idx+1:]
        if not os.path.isfile(im_path):
            QMessageBox.critical(None, 'File Not Found', im_path)
            return [], []
        imlist.append(lines[i+1][idx+1:])

    return imlist, seglist


img_exts = ['.nii', '.nii.gz', '.mhd', '.mha', '.hdr']


def find_image_names(folder, exts=img_exts):

    img_files = set()
    for case_name in os.listdir(folder):
        case_folder = os.path.join(folder, case_name)
        for root, folders, files in os.walk(case_folder):
            for file in files:
                file = os.path.basename(file)
                if any([file.endswith(ext) for ext in exts]):
                    name = os.path.join(root.split(case_folder)[1], file)
                    name = name.lstrip('/')
                    img_files.add(name)

    return img_files


class SegViewer(QWidget):
    """
    Segmentation viewer
    A convenient tool to view a list of intensity-segmentation image pairs
    """

    def __init__(self, gpu_id=0):

        super(SegViewer, self).__init__()

        self.cfg = view3_cfg.View3ConfigFactory().cfg_2x2_nosel('segviewer')
        self.imlist = []
        self.seglist = []
        self.sel_imset = {}
        self.auto_wcww = False
        self.wc = 0
        self.ww = 1000
        self.alpha = 0.25
        self.imidx = -1
        self.im3 = None
        self.seg3 = None
        self.auto_wc = None
        self.auto_ww = None
        self.seg_pad = 50

        self.segmodel = {}              # loaded segmentation model
        self.gpu_id = gpu_id            # which gpu to run model

        self.setAcceptDrops(True)
        self.setup_ui()

    def setup_ui(self):

        self.setWindowTitle('MD SegViewer')

        fontsize = 16
        font = self.font()
        font.setPointSize(fontsize)
        self.setFont(font)

        top_layout = QHBoxLayout()
        right_layout = QVBoxLayout()

        # file list path
        im_layout = QHBoxLayout()
        self.imEdit = QLineEdit(self)
        self.imEdit.setReadOnly(True)
        self.imButton = QPushButton('Image', self)

        self.imButton.clicked.connect(self.imButtonClicked)

        im_layout.addWidget(self.imEdit, 5)
        im_layout.addWidget(self.imButton, 1)

        # model folder path
        model_layout = QHBoxLayout()
        self.modelEdit = QLineEdit(self)
        self.modelEdit.setReadOnly(True)
        self.modelButton = QPushButton('Model', self)
        self.modelButton.clicked.connect(self.modelButtonClicked)

        model_layout.addWidget(self.modelEdit, 5)
        model_layout.addWidget(self.modelButton, 1)

        # segmentation button
        self.segButton = QPushButton('Segmentation', self)
        self.segButton.clicked.connect(self.segButtonClicked)

        # wc/ww box
        self.auto_box = QCheckBox('auto wc/ww', self)
        self.auto_box.stateChanged.connect(self.autoStateChanged)

        self.wc_text = QLabel('wc:')
        self.wc_edit = QLineEdit(self)
        self.wc_edit.setText('{}'.format(self.wc))
        self.wc_edit.editingFinished.connect(self.wcEditingFinished)

        wc_layout = QHBoxLayout()
        wc_layout.addWidget(self.wc_text)
        wc_layout.addWidget(self.wc_edit)

        self.ww_text = QLabel('ww:')
        self.ww_edit = QLineEdit(self)
        self.ww_edit.setText('{}'.format(self.ww))
        self.ww_edit.editingFinished.connect(self.wwEditingFinished)

        ww_layout = QHBoxLayout()
        ww_layout.addWidget(self.ww_text)
        ww_layout.addWidget(self.ww_edit)

        win_layout = QVBoxLayout()
        win_layout.addWidget(self.auto_box)
        win_layout.addLayout(wc_layout)
        win_layout.addLayout(ww_layout)

        # alpha
        self.alpha_box = QCheckBox('alpha')
        self.alpha_box.setChecked(True)
        self.alpha_box.stateChanged.connect(self.alphaClicked)

        self.alpha_bar = QSlider(Qt.Horizontal, self)
        self.alpha_bar.setSingleStep(1)
        self.alpha_bar.setPageStep(2)
        self.alpha_bar.setMinimum(0)
        self.alpha_bar.setMaximum(100)
        self.alpha_bar.setValue(25)
        self.alpha_bar.valueChanged.connect(self.alphaChanged)

        alpha_layout = QHBoxLayout()
        alpha_layout.addWidget(self.alpha_box)
        alpha_layout.addWidget(self.alpha_bar)

        # button
        ctrl_layout = QHBoxLayout()
        prev_button = QPushButton('prev', self)
        next_button = QPushButton('next', self)
        prev_button.clicked.connect(self.prevClicked)
        next_button.clicked.connect(self.nextClicked)

        ctrl_layout.addWidget(prev_button)
        ctrl_layout.addWidget(next_button)

        self.imlist_model = QStandardItemModel()
        self.imlist_view = QListView(self)
        self.imlist_view.setModel(self.imlist_model)
        self.imlist_view.doubleClicked.connect(self.itemDoubleClicked)
        self.imlist_view.installEventFilter(self)

        # add sub-layout
        right_layout.addLayout(im_layout)
        right_layout.addLayout(model_layout)
        right_layout.addWidget(self.segButton)
        right_layout.addLayout(win_layout)
        right_layout.addLayout(alpha_layout)
        right_layout.addLayout(ctrl_layout)
        right_layout.addWidget(self.imlist_view)

        self.view3 = View3(self.cfg)
        top_layout.addWidget(self.view3, 4)
        top_layout.addLayout(right_layout, 1)
        self.setLayout(top_layout)

    def update_listview(self):

        model = self.imlist_view.model()
        assert isinstance(model, QStandardItemModel)
        model.removeRows(0, model.rowCount())

        digits = int(np.ceil(np.log10(len(self.imlist))))
        format_str = '{:0' + str(digits) + 'd}'

        for i, file in enumerate(self.imlist):
            item = QStandardItem(format_str.format(i) + ' ' + file)
            item.setEditable(False)
            model.appendRow(item)

    def current_imname(self):

        digits = int(np.ceil(np.log10(len(self.imlist))))
        format_str = '{:0' + str(digits) + 'd}'

        imname = ('im_' + format_str).format(self.imidx)
        segname = ('seg_' + format_str).format(self.imidx)
        return imname, segname

    def read_imlist_from_file(self, filename):

        lines = readlines(filename)
        if len(lines) < 2:
            QMessageBox.critical(self, 'Error', 'empty/wrong file? too few lines')
            return

        self.read_imlist_from_lines(lines)

    def read_imlist_from_lines(self, lines):

        case_num = int(lines[0])
        if len(lines) >= case_num * 2 + 1:
            imlist, seglist = read_train_list(lines)
        elif len(lines) >= case_num + 1 and ' ' in lines[1]:
            imlist, seglist = read_test_list(lines)
        else:
            QMessageBox.critical(self, 'Error', 'empty/wrong file? incorrect format')
            return

        if len(imlist) == 0:
            return
        else:
            self.imlist, self.seglist = imlist, seglist
            self.sel_imset = set()

        self.imidx = -1
        self.im3 = None
        self.seg3 = None
        self.view3.reset_data()

        self.update_listview()
        self.view3.update_all()

    def read_train_folder(self, folder):

        names = find_image_names(folder)
        if len(names) < 2:
            QMessageBox.critical(self, 'Error', 'Less than two different image names')
        else:
            diag = TrainNameDialog()
            diag.org_combobox.addItems(names)
            diag.seg_combobox.addItems(names)
            diag.exec_()
            if diag.result() == QDialog.Accepted:
                with tempfile.TemporaryFile() as tmp:
                    make_train_imlist_fp(folder, tmp, diag.org_name, diag.seg_name)
                    tmp.seek(0)
                    linelist = tmp.readlines()
                for i in range(len(linelist)):
                    linelist[i] = linelist[i].rstrip('\n')
                self.read_imlist_from_lines(linelist)

    def read_test_folder(self, folder):

        names = find_image_names(folder)
        if len(names) == 0:
            QMessageBox.critical(self, 'Error', 'Empty folder')
        else:
            names = ['None'] + list(names)
            diag = TestNameDialog()
            diag.org_combobox.addItems(names)
            diag.exec_()
            if diag.result() == QDialog.Accepted:
                org_name = None if diag.org_name == 'None' else diag.org_name
                with tempfile.TemporaryFile() as tmp:
                    try:
                        make_test_imlist_fp(folder, tmp, org_name)
                    except Exception as e:
                        QMessageBox.warning(self, 'Warning', e.args[0])
                        return
                    tmp.seek(0)
                    linelist = tmp.readlines()
                for i in range(len(linelist)):
                    linelist[i] = linelist[i].rstrip('\n')
                self.read_imlist_from_lines(linelist)

    def compute_zoom_factor(self, world_size):

        spacing = self.view3.spacing

        factors = []
        view2 = self.view3.view2_group[0][0]
        width, height = view2.width(), view2.height()
        factors.append(world_size[0] / (width * spacing))
        factors.append(world_size[1] / (height * spacing))

        view2 = self.view3.view2_group[0][1]
        width, height = view2.width(), view2.height()
        factors.append(world_size[1] / (width * spacing))
        factors.append(world_size[2] / (height * spacing))

        view2 = self.view3.view2_group[0][2]
        width, height = view2.width(), view2.height()
        factors.append(world_size[0] / (width * spacing))
        factors.append(world_size[2] / (height * spacing))

        return np.max(factors)

    def auto_world_coord(self, im3):

        center = im3.center()
        self.view3.center3 = center
        self.view3.cursor3 = center

        spacing = im3.spacing()
        spacing = np.min(spacing)
        self.view3.spacing = spacing

        min_coord, max_coord = self.im3.world_box()
        world_size = max_coord - min_coord

        zoom_factor = self.compute_zoom_factor(world_size)
        self.view3.zoom_factor = zoom_factor
        self.view3.init_zoom_factor = zoom_factor

    def set_seg_lms(self, seg3):

        labels = ctools.count_labels(seg3)
        if len(labels) == 0:
            return

        minb, maxb = ctools.bounding_box_voxel_multi(seg3, labels)
        spacing = seg3.spacing()

        for label in labels:
            min_coord, max_coord = minb[label], maxb[label]
            box_size = (max_coord - min_coord) * spacing
            box_size += self.seg_pad * 2
            box_center = (max_coord + min_coord) / 2.0
            box_center = seg3.voxel_to_world(box_center)

            zoom_factor = self.compute_zoom_factor(box_size)
            self.view3.add_lm(str(label), box_center, zoom_factor, tag_visible=False)

    def load_set_single_image(self, filename):

        # reset first
        self.imlist = [filename]
        self.seglist = []
        self.sel_imset = set()

        self.imidx = -1
        self.im3 = None
        self.seg3 = None
        self.view3.reset_data()
        self.update_listview()
        self.view3.update_all()

        # view image
        self.imidx = 0
        idx = self.imlist_model.index(self.imidx, 0)
        self.imlist_view.setCurrentIndex(idx)
        self.view_image(0)

    def load_set_multiple_images(self, imlist):

        # reset data and view
        self.imlist = imlist
        self.seglist = []
        self.sel_imset = set()

        self.imidx = -1
        self.im3 = None
        self.seg3 = None
        self.view3.reset_data()

        self.update_listview()
        self.view3.update_all()

        # select and view the first image
        self.imidx = 0
        idx = self.imlist_model.index(self.imidx, 0)
        self.imlist_view.setCurrentIndex(idx)
        self.view_image(self.imidx)

    def load_set_single_seg(self, filename):

        imname, segname = self.current_imname()

        self.seglist = [filename]
        self.seg3 = cio.read_image(filename)
        self.view3.add_image(self.seg3, segname, 0, 1, 5, self.alpha)
        self.view3.select_images(0, [imname, segname])
        self.view3.update_all()

        self.set_seg_lms(self.seg3)

    def view_image(self, imidx):

        self.setWindowTitle('MD SegViewer - Loading Image {} ...'.format(imidx))

        # clean previous setting and data in view3
        self.view3.reset_data()

        digits = int(np.ceil(np.log10(len(self.imlist))))
        format_str = '{:0' + str(digits) + 'd}'

        imname = ('im_' + format_str).format(imidx)
        segname = ('seg_' + format_str).format(imidx)
        impath = self.imlist[imidx]
        segpath = self.seglist[imidx] if imidx < len(self.seglist) else None

        if impath in self.sel_imset:
            self.view3.rd_text = 'SELECTED'
        else:
            self.view3.rd_text = ''

        if os.path.isfile(impath):
            # read a volumetric image file
            self.im3 = cio.read_image(impath, dtype=np.float32)

        elif os.path.isdir(impath):
            # read image from dicom folder
            series_ids = cio.get_dicom_series(impath)
            if len(series_ids) == 0:
                QMessageBox.critical(self, 'Error', 'No series found in {}'.format(impath))
                return
            elif len(series_ids) == 1:
                self.im3, _ = cio.read_dicom_series(impath, dtype=np.float32)
            else:
                series_id, ok_pressed = QInputDialog.getItem(self, 'Select A Dicom Series', 'Dicom Series', series_ids, 0, False)
                if not ok_pressed:
                    return
                self.im3, _ = cio.read_dicom_series(impath, dtype=np.float32, series_name=series_id)

        else:
            QMessageBox.critical(self, 'Error', 'Path not exists: {}'.format(impath))
            return

        self.auto_world_coord(self.im3)

        if self.auto_wcww:
            minv, maxv = ctools.percentiles(self.im3, [0.001, 0.999])
            wc, ww = (minv + maxv) / 2.0, maxv - minv
            self.auto_wc, self.auto_ww = wc, ww
        else:
            wc, ww = self.wc, self.ww
            self.auto_wc, self.auto_ww = None, None

        self.view3.add_image(self.im3, imname, wc, ww, 0, 1 - self.alpha)

        if segpath is not None:
            self.seg3 = cio.read_image(segpath)
            self.view3.add_image(self.seg3, segname, 0, 1, 5, self.alpha)

        if segpath is None:
            self.view3.select_images(0, [imname])
            self.view3.update_all()
        else:
            self.view3.select_images(0, [imname, segname])
            self.view3.update_all()

            # set segmentation lms for quick browsing
            self.set_seg_lms(self.seg3)

        self.setWindowTitle('MD SegViewer')

        self.view3.clear_lines(0)

    def seg_image(self):

        if self.im3 is None:
            QMessageBox.warning(self, 'Warning', 'Empty image?')
            return

        if isinstance(self.segmodel, dict) and len(self.segmodel) == 0:
            QMessageBox.warning(self, 'Warning', 'Empty segmentation model?')
            return

        # segmentation
        empty_seg = self.seg3 is None
        self.seg3, box_center, box_size, test_time = autoseg_volume(self.im3, self.segmodel)

        # show segmentation on image
        digits = int(np.ceil(np.log10(len(self.imlist))))
        format_str = '{:0' + str(digits) + 'd}'
        segname = ('seg_' + format_str).format(self.imidx)

        self.view3.add_image(self.seg3, segname, 0, 1, 5, self.alpha)
        if empty_seg:
            self.view3.select_images(0, [segname])
        self.view3.update_all([segname])

        # goto organ center
        box_size += self.seg_pad * 2
        zoom_factor = self.compute_zoom_factor(box_size)
        self.view3.goto(box_center, zoom_factor)

        # print out segmentation time
        self.view3.clear_lines(0)
        self.view3.add_text_line(0, u'Auto-segmentation takes {:.2f} seconds'.format(test_time))

    def toggle_image(self):

        if self.imidx < 0 or self.imidx >= len(self.imlist):
            return

        impath = self.imlist[self.imidx]
        if impath in self.sel_imset:
            self.sel_imset.discard(impath)
            self.view3.rd_text = ''
        else:
            self.sel_imset.add(impath)
            self.view3.rd_text = 'SELECTED'

        self.view3.update_view(0)

    def export_selected_imlist(self):

        sel_imlist = list(self.sel_imset)
        if len(sel_imlist) == 0:
            QMessageBox.warning(self, 'Warning', 'No images selected')
            return

        filename = QFileDialog.getSaveFileName(self, 'Save Selected Imlist', filter='Imlist Files (*.txt)')
        if len(filename[0]) == 0:
            return

        with open(filename[0], 'w') as f:
            f.write('{}\n'.format(len(sel_imlist)))
            for i, impath in enumerate(sel_imlist):
                f.write('{} {}\n'.format(i, impath))

    # event handlers

    def imButtonClicked(self):

        diag = OptionDialog(['Imlist', 'Train', 'Test', 'Image', 'Dicom'], fontsize=16)
        diag.resize(750, 100)
        diag.exec_()

        if diag.selected_id < 0:
            return

        elif diag.selected_id == 0:
            # imlist
            filename = QFileDialog.getOpenFileName(self, 'Open Imlist File', filter='Imlist Files (*.txt)')
            if len(filename[0]) == 0:
                return
            self.imEdit.setText(filename[0])
            self.read_imlist_from_file(filename[0])

        elif diag.selected_id == 1:
            # train folder
            foldername = QFileDialog.getExistingDirectory(self, 'Open Image Folder', '', QFileDialog.ShowDirsOnly | QFileDialog.DontResolveSymlinks)
            if len(foldername) == 0:
                return
            self.imEdit.setText(foldername)
            self.read_train_folder(foldername)

        elif diag.selected_id == 2:
            # test folder
            foldername = QFileDialog.getExistingDirectory(self, 'Open Image Folder', '', QFileDialog.ShowDirsOnly | QFileDialog.DontResolveSymlinks)
            if len(foldername) == 0:
                return
            self.imEdit.setText(foldername)
            self.read_test_folder(foldername)

        elif diag.selected_id == 3:
            # single image/seg
            filename = QFileDialog.getOpenFileName(self, 'Open Image File', filter='Images (*.mhd *.mha *.hdr *.nii *.nii.gz)')
            if len(filename[0]) == 0:
                return
            self.imEdit.setText(filename[0])

            if self.im3 is None:
                self.load_set_single_image(filename[0])
            else:
                diag = OptionDialog(['Image', 'Seg'], fontsize=16)
                diag.resize(300, 100)
                diag.exec_()
                if diag.selected_id == 0:
                    self.load_set_single_image(filename[0])
                elif diag.selected_id == 1:
                    self.load_set_single_seg(filename[0])
                else:
                    return

        elif diag.selected_id == 4:
            # dicom data
            foldername = QFileDialog.getExistingDirectory(self, 'Open Dicom Folder', '', QFileDialog.ShowDirsOnly | QFileDialog.DontResolveSymlinks)
            if len(foldername) == 0:
                return
            self.imEdit.setText(foldername)
            self.load_set_single_image(foldername)

        else:
            QMessageBox.critical(self, 'Error', 'Unknown option')

    def modelButtonClicked(self):

        foldername = QFileDialog.getExistingDirectory(self, 'Open Model Folder',
                                                      options=QFileDialog.ShowDirsOnly | QFileDialog.DontResolveSymlinks)
        if len(foldername) == 0:
            return

        try:
            self.segmodel = autoseg_load_model(foldername, self.gpu_id)
            self.modelEdit.setText(foldername)
        except Exception as e:
            print e
            self.segmodel = {}
            self.modelEdit.setText('')
            QMessageBox.information(self, 'Info', 'Fail to load the segmentation model')

    def segButtonClicked(self):

        self.seg_image()

    def autoStateChanged(self, state):

        if state == Qt.Checked:
            self.ww_edit.setEnabled(False)
            self.wc_edit.setEnabled(False)
            self.auto_wcww = True

            if self.im3 is not None:

                if self.auto_ww is not None and self.auto_wc is not None:
                    # use cached value if possible
                    wc, ww = self.auto_wc, self.auto_ww
                else:
                    # compute wc, ww
                    minv, maxv = ctools.percentiles(self.im3, [0.001, 0.999])
                    wc, ww = (minv + maxv) / 2.0, maxv - minv
                    self.auto_wc, self.auto_ww = wc, ww

                self.wc, self.ww = wc, ww
                self.wc_edit.setText('{}'.format(wc))
                self.ww_edit.setText('{}'.format(ww))

                imname, _ = self.current_imname()
                self.view3.im_params[imname]['win_center'] = wc
                self.view3.im_params[imname]['win_width'] = ww

                for view2 in self.view3.view2_group[0]:
                    vidx = view2.vidx
                    self.view3.update_byte_imlist(vidx, [imname])
                    view2.image = self.view3.mix_im[vidx]
                    view2.update_view()

        else:
            self.ww_edit.setEnabled(True)
            self.wc_edit.setEnabled(True)
            self.auto_wcww = False

    def wcEditingFinished(self):

        edit = self.sender()
        try:
            self.wc = int(edit.text())

            if self.imidx >= 0:
                imname, _ = self.current_imname()
                self.view3.im_params[imname]['win_center'] = self.wc

                for view2 in self.view3.view2_group[0]:
                    vidx = view2.vidx
                    self.view3.update_byte_imlist(vidx, [imname])
                    view2.image = self.view3.mix_im[vidx]
                    view2.update_view()

        except:
            edit.setText('{}'.format(self.wc))
            QMessageBox.critical(self, 'Error', 'incorrect integer for wc')

    def wwEditingFinished(self):

        edit = self.sender()
        try:
            self.ww = int(edit.text())

            if self.imidx >= 0:
                imname, _ = self.current_imname()
                self.view3.im_params[imname]['win_width'] = self.ww

                for view2 in self.view3.view2_group[0]:
                    vidx = view2.vidx
                    self.view3.update_byte_imlist(vidx, [imname])
                    view2.image = self.view3.mix_im[vidx]
                    view2.update_view()

        except:
            edit.setText('{}'.format(self.ww))
            QMessageBox.critical(self, 'Error', 'incorrect integer for ww')

    def alphaClicked(self):

        if self.alpha_box.isChecked():
            self.alpha_bar.setEnabled(True)
            value = self.alpha_bar.value()
            self.alphaChanged(value)
        else:
            self.alpha_bar.setEnabled(False)
            self.alphaChanged(0)

    def flipAlphaCheckBox(self):

        if self.alpha_box.isChecked():
            self.alpha_box.setChecked(False)
            self.alpha_bar.setEnabled(False)
            self.alphaChanged(0)
        else:
            self.alpha_box.setChecked(True)
            self.alpha_bar.setEnabled(True)
            value = self.alpha_bar.value()
            self.alphaChanged(value)

    def alphaChanged(self, value):

        self.alpha = value / 100.0

        if self.imidx >= 0:
            imname, segname = self.current_imname()
            self.view3.im_params[imname]['alpha'] = 1 - self.alpha
            if segname in self.view3.im_params:
                self.view3.im_params[segname]['alpha'] = self.alpha

            for view2 in self.view3.view2_group[0]:
                vidx = view2.vidx
                self.view3.update_mix_image(vidx)
                view2.image = self.view3.mix_im[vidx]
                view2.update_view()

    def prevClicked(self):

        rows = self.imlist_model.rowCount()
        if rows == 0:
            return

        if self.imidx == 0:
            QMessageBox.information(self, 'Info', 'You have reached the beginning of image list')
            return

        self.imidx = (self.imidx - 1) % rows
        idx = self.imlist_model.index(self.imidx, 0)
        self.imlist_view.setCurrentIndex(idx)
        self.view_image(self.imidx)

    def nextClicked(self):

        rows = self.imlist_model.rowCount()
        if rows == 0:
            return

        if self.imidx == rows - 1:
            QMessageBox.information(self, 'Info', 'You have reached the end of image list')
            return

        self.imidx = (self.imidx + 1) % rows
        idx = self.imlist_model.index(self.imidx, 0)
        self.imlist_view.setCurrentIndex(idx)
        self.view_image(self.imidx)

    def itemDoubleClicked(self, idx):

        data = self.imlist_model.data(idx)
        idx = data.index(' ')
        self.imidx = int(data[:idx])
        self.view_image(self.imidx)

    def eventFilter(self, watched, event):

        if event.type() == QEvent.KeyPress and event.matches(QKeySequence.InsertParagraphSeparator):

            idx = self.imlist_view.currentIndex()
            self.imidx = idx.row()
            self.view_image(self.imidx)
            return True

        elif event.type() == QEvent.KeyPress and event.key() == Qt.Key_PageUp:

            self.prevClicked()
            return True

        elif event.type() == QEvent.KeyPress and event.key() == Qt.Key_PageDown:

            self.nextClicked()
            return True

        return False

    def keyPressEvent(self, event):

        assert isinstance(event, QKeyEvent)
        super(SegViewer, self).keyPressEvent(event)

        if event.key() == Qt.Key_PageDown:
            self.nextClicked()
            event.accept()
        elif event.key() == Qt.Key_PageUp:
            self.prevClicked()
            event.accept()
        elif event.key() == Qt.Key_X:
            self.flipAlphaCheckBox()
            event.accept()
        elif event.key() == Qt.Key_Space:
            self.toggle_image()
            event.accept()
        elif event.key() == Qt.Key_F12:
            self.export_selected_imlist()
            event.accept()
        else:
            self.view3.keyPressEvent(event)

    def dragEnterEvent(self, event):

        assert isinstance(event, QDragEnterEvent)
        if event.mimeData().hasUrls():
            event.acceptProposedAction()

    def dropEvent(self, event):

        assert isinstance(event, QDropEvent)
        mime_data = event.mimeData()
        url_list = mime_data.urls()

        imlist = []
        folder = None

        for url in url_list:
            file_path = url.toLocalFile()

            if os.path.isdir(file_path):
                folder = file_path
            elif(any([file_path.endswith(ext) for ext in img_exts])):
                imlist.append(file_path)

        if folder is not None and len(imlist) > 0:

            QMessageBox.warning(self, 'Warning', 'Please select either folder or files only')
            return

        elif folder is not None:

            # drag a folder
            diag = OptionDialog(['Train', 'Test'], fontsize=16)
            diag.resize(300, 100)
            diag.exec_()
            if diag.selected_id == 0:
                self.read_train_folder(folder)
            elif diag.selected_id == 1:
                self.read_test_folder(folder)
            else:
                return

        elif len(imlist) > 0:
            # drag multiple files or single file
            if self.im3 is None or len(imlist) > 1:

                self.load_set_multiple_images(imlist)

            else:

                diag = OptionDialog(['Image', 'Seg'], fontsize=16)
                diag.resize(300, 100)
                diag.exec_()
                if diag.selected_id == 0:
                    self.load_set_single_image(imlist[0])
                elif diag.selected_id == 1:
                    self.load_set_single_seg(imlist[0])
                else:
                    return

        else:
            event.accept()
            return


if __name__ == '__main__':

    import sys

    gpu_id = 0
    if len(sys.argv) > 1:
        gpu_id = int(sys.argv[1])

    app = QApplication(sys.argv)
    viewer = SegViewer(gpu_id)
    viewer.resize(1024, 800)
    viewer.show()
    sys.exit(app.exec_())
